In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from notebook_utils import NotebookUtilities
import os.path as osp
import os, re
import pandas as pd

nu = NotebookUtilities(data_folder_path=os.path.abspath('../data'))

In [3]:

for sub_directory, directories_list, files_list in os.walk('C:\\'):
    for file_name in files_list:
        if file_name.endswith('.docx') and ('resume' in file_name.lower()):
            file_path = os.path.join(sub_directory, file_name)
            if os.path.isfile(file_path):
                print(f'{file_path}')

C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume (1).docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_Cold_Fusion_Developer.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_Delivery_Driver.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_Jr._Java_Developer.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_Predictive_Analyst.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_QA_Tester.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_Reactor_Operator.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\Other\Job Hunting\Dave_Babbitt_Resume_Research_Analyst.docx
C:\Users\daveb\OneDrive\Documents\Google Drive\School\Predictive A

In [13]:

# Get a list of rogue functions already in scraping utilities
scraping_utils_set = nu.get_utility_file_functions(util_path='../py/scraping_utils.py')

# Make a set of rogue su functions
fn_regex = re.compile(r'\s+"def ([a-z0-9_]+)\(')
black_list = ['.ipynb_checkpoints', '$Recycle.Bin']
rows_list = []
this_folder = osp.dirname(osp.abspath(osp.curdir))
rogue_su_fns_set = set()
for sub_directory, directories_list, files_list in os.walk(this_folder):
    if all(map(lambda x: x not in sub_directory, black_list)):
        for file_name in files_list:
            if file_name.endswith('.ipynb'):
                file_path = osp.join(sub_directory, file_name)
                with open(file_path, 'r', encoding='utf-8') as f:
                    lines_list = f.readlines()
                    for line in lines_list:
                        match_obj = fn_regex.search(line)
                        if match_obj:
                            fn = match_obj.group(1)
                            if fn in scraping_utils_set: rogue_su_fns_set.add(fn)
if rogue_su_fns_set:
    print(f'Search for *.ipynb; file masks in the {this_folder} folder for this pattern:')
    print('\\s+"def (' + '|'.join(rogue_su_fns_set) + ')\(')
    print('Replace each of the calls to these definitions with calls the the su. equivalent (and delete the definitions).')

In [12]:

# Get a list of multi-notebook functions as notebook utilities candidates
fn_regex = re.compile(r'\s+"def ([a-z0-9_]+)\(')
black_list = ['.ipynb_checkpoints', '$Recycle.Bin']
rows_list = []
this_folder = osp.dirname(osp.abspath(osp.curdir))
for sub_directory, directories_list, files_list in os.walk(this_folder):
    if all(map(lambda x: x not in sub_directory, black_list)):
        for file_name in files_list:
            if file_name.endswith('.ipynb'):
                file_path = osp.join(sub_directory, file_name)
                with open(file_path, 'r', encoding='utf-8') as f:
                    lines_list = f.readlines()
                    fns_set = set()
                    for line in lines_list:
                        match_obj = fn_regex.search(line)
                        if match_obj:
                            fn = match_obj.group(1)
                            fns_set.add(fn)
                    for fn in fns_set:
                        row_dict = {}
                        row_dict['file_name'] = file_name
                        row_dict['function_name'] = fn
                        rows_list.append(row_dict)
function_names_df = pd.DataFrame(rows_list)
df = function_names_df.groupby('function_name').count().sort_values('file_name', ascending=False)
mask_series = (df.file_name > 1) & df.index.map(lambda x: len(x) > 4)
fns_list = df[mask_series].index.tolist()
if fns_list:
    print(f'Search for *.ipynb; file masks in the {this_folder} folder for this pattern:')
    print('\\s+"def (' + '|'.join(fns_list) + ')\(')
    print('Find a way to get rid of these definition duplicates.')

In [7]:

import os

for sub_directory, directories_list, files_list in os.walk(r'D:\\'):
    for file_name in files_list:
        if file_name.endswith('.svg'):
            file_path = os.path.join(sub_directory, file_name)
            if os.path.isfile(file_path):
                with open(file_path, 'r', errors='ignore') as file:
                    data = file.read()
                    if 'Retarded' in data:
                        print(f'{file_path}')

D:\\Geek Squad Data Backup\HDD\Documents\GitHub\notebooks_old\political_triangle\svg\nick_land_comments.svg
D:\\Geek Squad Data Backup\HDD\Documents\GitHub\notebooks_old\political_triangle\svg\nick_land_comments_copy.svg
D:\\Geek Squad Data Backup\HDD\Documents\GitHub\notebooks_old\political_triangle\svg\nick_land_comments_optimized.svg
D:\\Geek Squad Data Backup\HDD\Documents\GitHub\notebooks_old\political_triangle\svg\nick_land_comments_plain.svg
D:\\Geek Squad Data Backup\HDD\Documents\GitHub\notebooks_old\political_triangle\svg\.ipynb_checkpoints\nick_land_comments_optimized-checkpoint.svg


In [ ]:

import os

exe_list = ['wc', 'gvpr']
for sub_directory, directories_list, files_list in os.walk(r'D:\\'):
    for file_name in files_list:
        if any(map(lambda x: file_name == f'{x}.exe', exe_list)):
            file_path = os.path.join(sub_directory, file_name)
            print(f'{file_path}')

In [ ]:

%run ../load_magic/storage.py
%run ../load_magic/dataframes.py
%run ../load_magic/paths.py
%run ../load_magic/lists.py
%run ../load_magic/environment.py
%pprint
notebook_path = get_notebook_path()
print(notebook_path)
s = Storage()
print(['s.{}'.format(fn) for fn in dir(s) if not fn.startswith('_')])
[fn for fn in dir() if not fn.startswith('_')]